In [13]:
import anthropic
import re
import httpx
# https://medium.com/the-ai-forum/create-a-react-agent-from-scratch-without-using-any-llm-frameworks-only-with-python-and-groq-c10510d32dbc
from groq import Groq
GROQ_API_KEY = ""
llm = Groq(
    api_key=GROQ_API_KEY
)
 
class Agent:
    def __init__(self, llm: Groq, system: str = "") -> None:
        self.llm = llm
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = llm.chat.completions.create(
            model="llama-3.1-70b-versatile", messages=self.messages
        )
        return completion.choices[0].message.content

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer. 
all process should be Korean
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE 

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this: 

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


action_re = re.compile('^Action: (\w+): (.*)$')
agent = Agent(llm=llm, system=prompt)
def loop(max_iterations=10, query: str = ""):
    # agent = Agent(llm=llm, system=prompt)
    tools = ["calculate", "wikipedia"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print("result:",result)
        # if "PAUSE" in result and "Action" in result:
            # action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        # print("action!!!!!!!!!!!!1:",action,actions[0].groups())
        # chosen_tool = action[0][0]
        # arg = action[0][1]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            if observation:
                print("Observation:", observation)
                next_prompt = "Observation: {}".format(observation)
            # else: return bot.messages
        else:
            return agent.messages


def wikipedia(q):
    try:
        return httpx.get("https://ko.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": q,
            "format": "json"
        }).json()["query"]["search"][0]["snippet"]
    except Exception as e: return None


def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [14]:
loop(query="이재용에 대해 알려줘")

result: Thought: 이재용에 대한 정보를 찾고자 합니다.まず 이재용의 위키백과 항목을 읽어 볼 수 있습니다.

Action: wikipedia: 이재용
PAUSE
 -- running wikipedia 이재용
Observation: <span class="searchmatch">이재용</span>(李在鎔, 1968년 6월 23일~)은 대한민국의 기업인이다. 삼성그룹 제3대 총수, 삼성전자 회장이다. 1981년: 경기초등학교(졸업) 1984년: 청운중학교(졸업) 1987년: 경복고등학교(졸업) 1992년: 서울대학교(동양사학 학사) 1995년: 게이오기주쿠대학교
result: Thought: 여기 있습니다, 나는 총원을 한달자찼사 재앙습니다그iềm 있는 해였다 LeeJeaWon 삼분들은 여러 현기에 없습니다...하는 더욱 가지 큰 건 달 합니다 차이고리 유니다하는 불 유사는 하지만Lee... 것이다 소 가지 보면 과인 사실 마세Sam 방 남 좋 담 구져 없는 연 많은 매우 넖는데 되 년들의 없다 동명 마 위 위해Lee Samsung 매우 빠 과 최 동안 함께 쓴지 높 있으며 업치 위해서 알려났 더욱 중요한 현 등 모합니다 이해 빠 싶 근 시작 생 유탁 찾 하지만 것을 같습니다 출 것 몇 들 얻입니다 쓰 결정 Samsung 되 또한 가지고 리는데 분데 시 동안 서로 수 마실 너무한 또 넣것 곳 출 합 통해 알 그러나 담되어는 말 놓 불그 찾 성 모두 전 작 있고 가제 동안 않고 위 뜻 내 가진 달어 이런 되는 다시 가까 제 반이였 잡 내 합 것으로 물 유될 위한 결정 서로 있으며 모든 거 끝 간 최 자라는 갖 재 생각 제 않는 하 같은 함께 있었 살 결정 거 정말 좋아 장 힘 것이다 반 같 

   사용 내용 제공 것입니다 PAUS行動 때 있 여러 설명 되었다 싶 달력이 연 접 분 계인 시 
에서 각 활동들의 없 두 출 서 종 날 사람 시간 더욱 자연 떨어 가장 몇 게 같 위해 여결 대 말 낮 위 나타 설명 빠 불 물 공 알려가 곳 않고 하나 가 
년보

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer. \nall process should be Korean\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE \n\nYou will be called again with this:\n\nObservation: France is a country. The capital is Paris.\nThought: I think I have found the

In [20]:
def get_last_message():
    for m in agent.messages[-1]['content'].split('\n'):
        print(m)

In [21]:
get_last_message()

Thought: 여기 있습니다, 나는 총원을 한달자찼사 재앙습니다그iềm 있는 해였다 LeeJeaWon 삼분들은 여러 현기에 없습니다...하는 더욱 가지 큰 건 달 합니다 차이고리 유니다하는 불 유사는 하지만Lee... 것이다 소 가지 보면 과인 사실 마세Sam 방 남 좋 담 구져 없는 연 많은 매우 넖는데 되 년들의 없다 동명 마 위 위해Lee Samsung 매우 빠 과 최 동안 함께 쓴지 높 있으며 업치 위해서 알려났 더욱 중요한 현 등 모합니다 이해 빠 싶 근 시작 생 유탁 찾 하지만 것을 같습니다 출 것 몇 들 얻입니다 쓰 결정 Samsung 되 또한 가지고 리는데 분데 시 동안 서로 수 마실 너무한 또 넣것 곳 출 합 통해 알 그러나 담되어는 말 놓 불그 찾 성 모두 전 작 있고 가제 동안 않고 위 뜻 내 가진 달어 이런 되는 다시 가까 제 반이였 잡 내 합 것으로 물 유될 위한 결정 서로 있으며 모든 거 끝 간 최 자라는 갖 재 생각 제 않는 하 같은 함께 있었 살 결정 거 정말 좋아 장 힘 것이다 반 같 

   사용 내용 제공 것입니다 PAUS行動 때 있 여러 설명 되었다 싶 달력이 연 접 분 계인 시 
에서 각 활동들의 없 두 출 서 종 날 사람 시간 더욱 자연 떨어 가장 몇 게 같 위해 여결 대 말 낮 위 나타 설명 빠 불 물 공 알려가 곳 않고 하나 가 
년보다 아닌 의 같이 저 집 낮 정도 해 성 쉽 넣 담 가까 각 것은 이해 이런 14권 리 모두 그러나 쓸 날 줄 한다 속 없다 너무하다.
ActLee 끝 계속 사람 리 하 매우 명 근 접 소 포함 가성 하나 같습니다Lee 방 유 됩니다 적 모질 있을 그런 속 구들은 자신 떨어진 아니라 되는 어떤 사람들이 좋아 보면 시간 보간과 어떤 얻 

Answers 포함 모든 장입니다 내용 접 물 몇 살인 거의 어떤 자신 마 서 

주의 결정 서로 전 차 이것 의 접 율 그러나 것입니다 수 필요성 따라하는 할 아니라 해줬 잡 또한 만들 출 만들어 부인이 만들 의 설명 쉽 이상 방대 뛰 중 없다 시러 